In [199]:
from gemma import gm

# Model and parameters
# model = gm.nn.Gemma3_4B()
params = gm.ckpts.load_params("/Users/sebastianamenabar/Downloads/gemma-3-flax-gemma3-1b-it-int4-v1/gemma3-1b-it-int4/")

In [21]:
upper = params['layer_0']['mlp']['linear'].max(axis=0)
lower = params['layer_0']['mlp']['linear'].min(axis=0)

In [38]:
params.keys()

dict_keys(['embedder', 'final_norm', 'layer_0', 'layer_1', 'layer_10', 'layer_11', 'layer_12', 'layer_13', 'layer_14', 'layer_15', 'layer_16', 'layer_17', 'layer_18', 'layer_19', 'layer_2', 'layer_20', 'layer_21', 'layer_22', 'layer_23', 'layer_24', 'layer_25', 'layer_3', 'layer_4', 'layer_5', 'layer_6', 'layer_7', 'layer_8', 'layer_9'])

In [81]:
import jax.tree_util as tree_util
import numpy as onp
import mlx.core as mx

In [71]:
tree_util.tree_leaves_with_path(params)

[((DictKey(key='embedder'), DictKey(key='input_embedding')),
  Array([[ 0.        , -0.04347873,  0.        , ...,  0.04347873,
           0.        , -0.04347873],
         [ 0.05060371, -0.0168679 ,  0.        , ..., -0.0168679 ,
           0.0168679 ,  0.03373581],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [ 0.        , -0.04515978,  0.        , ...,  0.04515978,
           0.        , -0.04515978],
         [ 0.        , -0.04516123,  0.        , ...,  0.04516123,
           0.        , -0.04516123],
         [ 0.        , -0.04404405,  0.        , ...,  0.04404405,
           0.        , -0.04404405]], dtype=float32)),
 ((DictKey(key='final_norm'), DictKey(key='scale')),
  Array([ 5.168115 , 20.004885 ,  2.7891512, ...,  2.9477396, 14.478675 ,
          7.953793 ], dtype=float32)),
 ((DictKey(key='layer_0'),
   DictKey(key='attn'),
   DictKey(key='_key_norm'),
   DictKey(key='scale')),
  Array([ 0

In [82]:
def convert_quantized_linear(source):
    w = mx.array(onp.array(source).astype(onp.float16))
    wq, s, b = mx.quantize(source.T, group_size=source.shape[0])
    
    mx.async_eval(wq, s, b)
    return wq, s, b

In [207]:
mlx_format_weights = {}
for i in range(26):
    mlx_format_weights[f"model.layers.{i}.self_attn.k_norm.weight"] = (
        onp.array(params[f"layer_{i}"]["attn"]["_key_norm"]["scale"]).astype(onp.float16)
    )
    mlx_format_weights[f"model.layers.{i}.self_attn.q_norm.weight"] = (
        onp.array(params[f"layer_{i}"]["attn"]["_query_norm"]["scale"]).astype(onp.float16)
    )
    mlx_format_weights[f"model.layers.{i}.input_layernorm.weight"] = (
        onp.array(params[f"layer_{i}"]["pre_attention_norm"]["scale"]).astype(onp.float16)
    )
    mlx_format_weights[f"model.layers.{i}.post_attention_layernorm.weight"] = (
        onp.array(params[f"layer_{i}"]["post_attention_norm"]["scale"]).astype(onp.float16)
    )
    mlx_format_weights[f"model.layers.{i}.pre_feedforward_layernorm.weight"] = (
        onp.array(params[f"layer_{i}"]["pre_ffw_norm"]["scale"]).astype(onp.float16)
    )
    mlx_format_weights[f"model.layers.{i}.post_feedforward_layernorm.weight"] = (
        onp.array(params[f"layer_{i}"]["post_ffw_norm"]["scale"]).astype(onp.float16)
    )

    quantized_linears = {}
    quantized_linears["self_attn.k_proj"] = (
        params[f"layer_{i}"]["attn"]["kv_einsum"]["w"][0, 0].T
    )
    quantized_linears["self_attn.v_proj"] = (
        params[f"layer_{i}"]["attn"]["kv_einsum"]["w"][1, 0].T
    )
    quantized_linears["self_attn.q_proj"] = (
        params[f"layer_{i}"]["attn"]["q_einsum"]["w"].transpose(0, 2, 1).reshape(1024, -1)
    )
    quantized_linears["self_attn.o_proj"] = (
        params[f"layer_{i}"]["attn"]["attn_vec_einsum"]["w"].reshape(1024, -1).T
    )

    quantized_linears["mlp.down_proj"] = (
        params[f"layer_{i}"]["mlp"]["linear"].T
    )
    quantized_linears["mlp.gate_proj"] = (
        params[f"layer_{i}"]["mlp"]["gating_einsum"][0]
    )

    quantized_linears["mlp.up_proj"] = (
        params[f"layer_{i}"]["mlp"]["gating_einsum"][1]
    )
    for target_name, w in quantized_linears.items():
        u = jnp.unique(w, size=17, fill_value=-1000)
        assert u <= 16
        w = mx.array(onp.asarray(w).astype(onp.float32))
        b = w.min(0)
        uw = w - b
        # wq, s, b = mx.quantize(w, group_size=128)
        assert (w == mx.dequantize(wq, s, b, 128)).all()
        # mlx_format_weights[target_name + ".weight"] = wq
        # mlx_format_weights[target_name + ".scales"] = s
        # mlx_format_weights[target_name + ".biases"] = b

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [228]:
jnp.unique(w[1])

Array([-0.09471427, -0.06765305, -0.05412244, -0.04059183, -0.02706122,
       -0.01353061,  0.        ,  0.01353061,  0.02706122,  0.04059183,
        0.05412244,  0.06765305,  0.08118366], dtype=float32)

In [224]:
u.shape

(17, 1152)

In [234]:
u

Array([[-0.08639986, -0.07405703, -0.07405703, -0.07405703, -0.07405703,
        -0.06171419, -0.06171419, -0.06171419, -0.06171419, -0.06171419,
        -0.06171419, -0.04937135, -0.04937135, -0.04937135, -0.04937135,
        -0.04937135, -0.04937135],
       [-0.06765305, -0.09471427, -0.06765305, -0.04059183, -0.02706122,
        -0.06765305, -0.05412244, -0.04059183, -0.02706122, -0.01353061,
         0.        , -0.05412244, -0.04059183, -0.02706122, -0.01353061,
         0.        ,  0.01353061]], dtype=float32)

In [233]:
u = jnp.unique(w[:2], axis=1, size=17, fill_value=-1000)

In [190]:
_w = (w - w.min(0))
(_w / _w.max(0)) * 16

array([[9.66431, 10.8352, 9.50331, ..., 7.48393, 6.23238, 9.22661],
       [9.66431, 10.1204, 6.91172, ..., 9.12002, 5.06308, 8.23994],
       [9.66431, 7.97471, 10.5356, ..., 7.00264, 7.21296, 8.96076],
       ...,
       [9.66431, 10.5647, 9.21696, ..., 9.55839, 12.09, 10.3724],
       [13.0328, 10.6841, 7.82928, ..., 8.26442, 5.4392, 9.65771],
       [9.66431, 6.74517, 7.82928, ..., 4.55181, 8.76776, 8.97415]], dtype=float32)

In [191]:
w.max(0) - w.min(0)

array([0.212102, 0.249686, 0.23594, ..., 0.253028, 0.201397, 0.45809], dtype=float32)

In [150]:
w.shape, wq.shape, s.shape, b.shape

((256, 1152), (256, 144), (256, 9), (256, 9))

In [194]:
u = onp.unique(onp.array(w[1]))

In [196]:
u

array([-0.09471427, -0.06765305, -0.05412244, -0.04059183, -0.02706122,
       -0.01353061,  0.        ,  0.01353061,  0.02706122,  0.04059183,
        0.05412244,  0.06765305,  0.08118366], dtype=float32)

In [204]:
((um - um.min()) / 0.17589793) * 13

array([ 0.       ,  1.9999999,  3.       ,  4.       ,  4.9999995,
        6.       ,  7.0000005,  8.       ,  9.       ,  9.999999 ,
       10.999999 , 12.       , 13.       ], dtype=float32)

In [187]:
um / onp.abs(u).max() * 8

array([ 0.   ,  2.29 ,  3.43 ,  4.57 ,  5.715,  6.855,  8.   ,  9.14 ,
       10.28 , 11.42 , 12.57 , 13.71 , 14.86 ], dtype=float16)

In [195]:
um = ((u - u.min()))

In [200]:
um[1:] - um[:-1]

array([0.02706122, 0.01353061, 0.01353061, 0.0135306 , 0.01353061,
       0.01353061, 0.01353061, 0.01353061, 0.0135306 , 0.01353061,
       0.01353061, 0.01353061], dtype=float32)

In [121]:
w = params["layer_0"]["attn"]["q_einsum"]["w"].transpose(0, 2, 1).reshape(1024, -1)

In [123]:
jnp.unique(w[0, :])

Array([-0.08821857, -0.07561591, -0.06301326, -0.05041061, -0.03780796,
       -0.0252053 , -0.01260265,  0.        ,  0.01260265,  0.0252053 ,
        0.03780796,  0.05041061,  0.06301326], dtype=float32)

In [83]:
total_params = 0
for k, v in tree_util.tree_leaves_with_path(params):
    print(".".join(map(lambda x: x.key, k)), v.size, v.shape)
    total_params += v.size

embedder.input_embedding 301989888 (262144, 1152)
final_norm.scale 1152 (1152,)
layer_0.attn._key_norm.scale 256 (256,)
layer_0.attn._query_norm.scale 256 (256,)
layer_0.attn.attn_vec_einsum.w 1179648 (4, 256, 1152)
layer_0.attn.kv_einsum.w 589824 (2, 1, 1152, 256)
layer_0.attn.q_einsum.w 1179648 (4, 1152, 256)
layer_0.mlp.gating_einsum 15925248 (2, 6912, 1152)
layer_0.mlp.linear 7962624 (6912, 1152)
layer_0.post_attention_norm.scale 1152 (1152,)
layer_0.post_ffw_norm.scale 1152 (1152,)
layer_0.pre_attention_norm.scale 1152 (1152,)
layer_0.pre_ffw_norm.scale 1152 (1152,)
layer_1.attn._key_norm.scale 256 (256,)
layer_1.attn._query_norm.scale 256 (256,)
layer_1.attn.attn_vec_einsum.w 1179648 (4, 256, 1152)
layer_1.attn.kv_einsum.w 589824 (2, 1, 1152, 256)
layer_1.attn.q_einsum.w 1179648 (4, 1152, 256)
layer_1.mlp.gating_einsum 15925248 (2, 6912, 1152)
layer_1.mlp.linear 7962624 (6912, 1152)
layer_1.post_attention_norm.scale 1152 (1152,)
layer_1.post_ffw_norm.scale 1152 (1152,)
layer_1.pr

In [75]:
total_params

999885952

In [66]:
".".join(map(lambda x: x.key, leaves[0][0]))

'embedder.input_embedding'

In [70]:
tree_util.tree_flatten(tree_util.tree_map_with_path((lambda k, v: (k, v.size)), params))

([DictKey(key='embedder'),
  DictKey(key='input_embedding'),
  301989888,
  DictKey(key='final_norm'),
  DictKey(key='scale'),
  1152,
  DictKey(key='layer_0'),
  DictKey(key='attn'),
  DictKey(key='_key_norm'),
  DictKey(key='scale'),
  256,
  DictKey(key='layer_0'),
  DictKey(key='attn'),
  DictKey(key='_query_norm'),
  DictKey(key='scale'),
  256,
  DictKey(key='layer_0'),
  DictKey(key='attn'),
  DictKey(key='attn_vec_einsum'),
  DictKey(key='w'),
  1179648,
  DictKey(key='layer_0'),
  DictKey(key='attn'),
  DictKey(key='kv_einsum'),
  DictKey(key='w'),
  589824,
  DictKey(key='layer_0'),
  DictKey(key='attn'),
  DictKey(key='q_einsum'),
  DictKey(key='w'),
  1179648,
  DictKey(key='layer_0'),
  DictKey(key='mlp'),
  DictKey(key='gating_einsum'),
  15925248,
  DictKey(key='layer_0'),
  DictKey(key='mlp'),
  DictKey(key='linear'),
  7962624,
  DictKey(key='layer_0'),
  DictKey(key='post_attention_norm'),
  DictKey(key='scale'),
  1152,
  DictKey(key='layer_0'),
  DictKey(key='post_f

In [22]:
upper, lower

(Array([0.04642943, 0.04865127, 0.05569537, ..., 0.05234581, 0.04807378,
        0.05065921], dtype=float32),
 Array([-0.05416766, -0.04865127, -0.04773889, ..., -0.04486784,
        -0.0412061 , -0.05065921], dtype=float32))

In [26]:
import jax.numpy as jnp

In [47]:
params['layer_0']['mlp']['linear'].nonzero()

(Array([   0,    0,    0, ..., 6911, 6911, 6911], dtype=int32),
 Array([   0,    2,    3, ..., 1144, 1146, 1150], dtype=int32))

In [33]:
jnp.unique(((params['layer_0']['mlp']['linear'][:, 3])))

Array([-0.04702289, -0.04030534, -0.03358778, -0.02687022, -0.02015267,
       -0.01343511, -0.00671756,  0.        ,  0.00671756,  0.01343511,
        0.02015267,  0.02687022,  0.03358778], dtype=float32)

In [28]:
jnp.unique(((params['layer_0']['mlp']['linear'][0] - lower[0]) / (upper[0] - lower[0])))

Array([-0.14086023, -0.14085484,  0.20556122,  0.22598866,  0.26222122,
        0.27092877,  0.28673244,  0.291529  ,  0.29681888,  0.29935315,
        0.30064106,  0.30938172,  0.33060393,  0.33365393,  0.33493364,
        0.33639097,  0.33811378,  0.3386682 ,  0.33957273,  0.33969763,
        0.34165734,  0.34813008,  0.3518619 ,  0.36094865,  0.3679527 ,
        0.3689025 ,  0.36946952,  0.36986306,  0.37141815,  0.37143153,
        0.3725076 ,  0.3733182 ,  0.37470302,  0.37507313,  0.37570924,
        0.37791765,  0.3781008 ,  0.37969503,  0.3805189 ,  0.38104412,
        0.38490415,  0.38742608,  0.38767642,  0.38906926,  0.38929182,
        0.38958156,  0.3898073 ,  0.39224568,  0.3930482 ,  0.39357367,
        0.39405364,  0.39458004,  0.39485297,  0.39505285,  0.39682505,
        0.39730418,  0.3973546 ,  0.39835033,  0.3989312 ,  0.39896798,
        0.3990987 ,  0.4005511 ,  0.40055543,  0.40093076,  0.4012484 ,
        0.4014011 ,  0.40176976,  0.40199506,  0.40205705,  0.40

In [ ]:
model = gm.nn.Gemma3_1B()